# 4.1- NLP

NLP trata de aplicaciones que entiendan nuestro idioma, reconocimiento de voz, traducción, comprensión semántica, análisis de sentimiento..

**Usos**

+ Motores de búsqueda
+ Feed de redes sociales
+ Asistentes de voz 
+ Filtros de span
+ Chatbots

**Librerías**

+ NLTK
+ Spacy
+ TFIDF
+ OpenNLP

La dificultad del NLP está en varios niveles:

+ Ambigüedad:

  * Nivel léxico: por ejemplo, varios significados
  * Nivel referencial: anáforas, metáforas, etc...
  * Nivel estructural: la semántica es necesaria para entender la estructura de una oración
  * Nivel pragmático: dobles sentidos, ironía, humor
  
+ Detección de espacios
+ Recepción imperfecta: acentos, -ismos, OCR

El proceso es similar que en USL, primero se vectorizan las palabras y después se miden sus distancias/similitudes. 

In [ ]:
# lista de 100 peliculas

titles=open('data/title_list.txt').read().split('\n')[:100]

titles[:10]

In [ ]:
synopsis=open('data/synopses_list.txt').read().split('\n BREAKS HERE')[:100]

synopsis[0][:100]

### Limpieza

In [ ]:
#!pip install spacy

In [ ]:
import string

import spacy

from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

import re

In [ ]:
#!python -m spacy download en

In [ ]:
nlp=spacy.load('en')

parser=English()

In [ ]:
def spacy_tokenizer(frase):
    
    tokens=parser(frase)
    #print(help(tokens))   # lo tratamos como string pero es un objeto de spacy
    
    clean_tokens=[]
    
    for e in tokens:
        
        lema=e.lemma_.lower().strip()
        
        if lema not in STOP_WORDS and re.search('^[a-zA-Z]+$', lema):
            
            clean_tokens.append(lema)
            
    return clean_tokens

In [ ]:
spacy_tokenizer(synopsis[0][:200])

### TFIDF (term frequency inverse document frequency)

In [ ]:
type(synopsis[0])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf=TfidfVectorizer(min_df=0.15, tokenizer=spacy_tokenizer)

In [ ]:
tfidf_matrix=tfidf.fit_transform(synopsis)

In [ ]:
tfidf_matrix.shape, len(synopsis)

In [ ]:
(str(tfidf_matrix[0]).split('\n'))

In [ ]:
import pandas as pd

df=pd.DataFrame(tfidf_matrix)

In [ ]:
df.head()

In [ ]:
terms=tfidf.get_feature_names()

terms[:15]

In [ ]:
# os dejo la kata

tfidf_matrix

### Distancias

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity as cos

In [ ]:
distancias=1-cos(tfidf_matrix)

distancias.shape

In [ ]:
pd.DataFrame(distancias).head()

### Clustering

In [ ]:
import warnings
warnings.simplefilter('ignore')

import pylab as plt
from IPython.display import set_matplotlib_formats

%matplotlib inline
set_matplotlib_formats('svg')

import numpy as np

In [ ]:
from umap import UMAP

In [ ]:
umap=UMAP(n_neighbors=5, random_state=42)

emb=umap.fit_transform(distancias)

In [ ]:
emb[:5]

In [ ]:
emb.shape

In [ ]:
plt.scatter(emb[:, 0], emb[:, 1]);

In [ ]:
from hdbscan import HDBSCAN

In [ ]:
hdbscan=HDBSCAN(min_cluster_size=5)

clusters=hdbscan.fit_predict(emb)

In [ ]:
np.unique(clusters)

In [ ]:
plt.scatter(emb[:, 0], emb[:, 1], c=clusters);

### titulos de los clusters

In [ ]:
def get_titles_from_cluster(c):
    return pd.Series(titles)[clusters==c]

In [ ]:
get_titles_from_cluster(2)

In [ ]:
tfidf_df=pd.DataFrame(tfidf_matrix.toarray(), columns=terms)

tfidf_df.head()

In [ ]:
def get_df_from_cluster(c):
    return tfidf_df[clusters==c]

In [ ]:
get_df_from_cluster(2).describe()

In [ ]:
get_df_from_cluster(2).describe().T

In [ ]:
top_words=get_df_from_cluster(1).T.sum(axis=1).sort_values(ascending=False)

top_words.head()

In [ ]:
get_titles_from_cluster(1)

## NLP_es 

In [ ]:
#!python -m spacy download es_core_news_md

In [ ]:
nlp=spacy.load('es_core_news_md')

In [ ]:
doc2vec=nlp('hola me llamo pepito').vector.sum()

doc2vec

In [ ]:
word2vec=(nlp('hola').vector + nlp('me').vector + nlp('llamo').vector + nlp('pepito').vector).sum()/4

word2vec

##### similitud

In [ ]:
nlp('quiero saludar a todos los alumnos de Ironhack, que pasa alegres').similarity(nlp('hola a todos los alumnos'))

In [ ]:
token_1=spacy_tokenizer('quiero saludar a todos los alumnos de Ironhack, que pasa alegres')
token_2=spacy_tokenizer('hola a todos los alumnos')

token_1

In [ ]:
def get_simil(t1, t2):
    return nlp(' '.join(t1)).similarity(nlp(' '.join(t2)))  # similitud es entre -1 y 1, por el valor del coseno

In [ ]:
get_simil(token_1, token_2)

# WordClouds

In [ ]:
#!pip install langdetect
#!pip install wordcloud

In [ ]:
# librerias

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image

import string
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

from spacy.lang.es import Spanish



import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words_sp=set(stopwords.words('spanish'))
stop_words_en=set(stopwords.words('english'))

stop_words=stop_words_sp | stop_words_en


import re

from langdetect import detect

from wordcloud import WordCloud

In [ ]:
def tokenizer(frase):
    
    if detect(frase)=='en':  # si esta en ingles...
        nlp=spacy.load('en')
        parser=English()
        
    elif detect(frase)=='es': # si esta en castellano...
        nlp=spacy.load('es_core_news_md')
        parser=Spanish()
    else:
        return 'No es ni castellano ni ingles..'
    
    
    tokens=parser(frase)
    
    clean_tokens=[]
    
    for e in tokens:
        
        lema=e.lemma_.lower().strip()
        
        if lema not in stop_words and re.search('^[a-zA-Z]+$', lema):
            
            clean_tokens.append(lema)
            
    return ' '.join(clean_tokens)

In [ ]:
def wordcloud(df, col):
    
    wordcloud=WordCloud(width=1600,
                        height=400,
                        stopwords=stop_words,
                        colormap='Spectral').generate(' '.join([e for e in df[col]]))
    
    plt.figure(figsize=(15, 10), facecolor='k')
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.tight_layout(pad=0)
    plt.savefig('images/wordcloud.png', facecolor='k', bbox_inches='tight')
    plt.show();

In [ ]:
df=pd.DataFrame(synopsis, columns=['text'])

df.text=df.text.apply(tokenizer)

df.head()

In [ ]:
wordcloud(df, 'text')

In [ ]:
txt=open('data/conde.txt').read().split('\n BREAKS HERE')[:100]

df2=pd.DataFrame(txt, columns=['text'])

df2.text=df2.text.apply(tokenizer)

df2.head()

In [ ]:
wordcloud(df2, 'text')

In [ ]:
#help(WordCloud)

#### Mascara

In [ ]:
# imagen con mascara

Image.open('images/vino.png')

In [ ]:
vino_mask=np.array(Image.open('images/vino.png'))

vino_mask.shape

In [ ]:
# transformacion de la mascara

def transformacion(val):
    if val==0:
        return 255
    else:
        return val

In [ ]:
t_vino_mask=np.ndarray((vino_mask.shape[0], vino_mask.shape[1]), np.int32)


for i in range(len(vino_mask)):
    t_vino_mask[i]=list(map(transformacion, vino_mask[i]))

In [ ]:
w=WordCloud(background_color='white',
                   max_words=1000,
                   mask=t_vino_mask,
                   stopwords=stop_words,
                   contour_width=3,
                   contour_color='firebrick').generate(' '.join([e for e in df.text]))


w.to_file('images/copa&botella.png')


plt.figure(figsize=(15, 10), facecolor='k')
plt.imshow(w)
plt.axis('off')
plt.tight_layout(pad=0)


### ejemplo con todo

In [ ]:
def tokenizer(lst):  # ahora entra una lista
    
    en=0
    es=0
    
    for txt in lst:
        try:
            txt=str(txt)
            if detect(txt)=='en':    # si el texto esta en ingles...
                en+=1
                
                nlp=spacy.load('en')
                parser=English()
                tokens=parser(txt)

                tokens_en=[]

                for word in tokens:
                    lemma=word.lemma_.lower().strip()
                    if lemma not in STOP_WORDS and re.search('^[a-zA-Z]+$', lemma):
                        tokens_en.append(lemma)

            elif detect(txt)=='es':   # si el texto esta en castellano...
                es+=1
                
                nlp=spacy.load('es_core_news_md')
                parser=Spanish()
                tokens=parser(txt)

                tokens_es=[]

                for word in tokens:
                    lemma=word.lemma_.lower().strip()
                    if lemma not in STOP_WORDS and re.search('^[a-zA-Z]+$', lemma):
                        tokens_es.append(lemma)

            else:
                print ('No se reconoce idioma (EN / ES)...')
        
        except:
            print ('ERROR...')
            continue
    
    
    return ' '.join(tokens_en), ' '.join(tokens_es), en, es

In [ ]:
mix_txt=synopsis+txt

tokens=tokenizer(mix_txt)

In [ ]:
# wordcloud en ingles

serie_en=pd.DataFrame({'en': tokens[0]}, index=[0])

wordcloud(serie_en, 'en')

In [ ]:
# wordcloud en castellano

serie_es=pd.DataFrame({'es': tokens[1]}, index=[0])

wordcloud(serie_es, 'es')

In [ ]:
tokens[2]

In [ ]:
tokens[3]

## NER

In [ ]:
news=pd.read_csv('data/noticias.csv')

news.head()

In [ ]:
news.tail(2).T

In [ ]:
import spacy
from spacy import displacy

In [ ]:
#!python -m spacy download es_core_news_lg

In [ ]:
#!python -m spacy download en_core_web_lg

In [ ]:
spacy_core_es='es_core_news_lg'

spacy_core_en='en_core_web_lg'

In [ ]:
def ner(spacy_core, data):
    
    nlp=spacy.load(spacy_core)
    
    frases=list(nlp(data).sents)  # frases, sentencias
    
    entidades=displacy.render(nlp(str(frases)), style='ent')
    
    return entidades

In [ ]:
ner(spacy_core_es, news.text[150])

In [ ]:
ner(spacy_core_en, synopsis[3][:1000])

### Transformers (creacion de texto)

In [ ]:
#!pip install transformers
#!pip install tensorflow
#!pip install torch

In [1]:
from transformers import pipeline

generador=pipeline('text-generation', 
                   model='EleutherAI/gpt-neo-125M')

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/560 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/357 [00:00<?, ?B/s]

In [2]:
def crea_texto(generador, texto, min_long=100):
    
    return generador(texto, do_sample=True, min_length=min_long)[0]['generated_text']

In [3]:
crea_texto(generador, 'data science is')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'data science is used as a way of getting more data and, thus, to predict whether a target'

In [4]:
crea_texto(generador, 'what planet is this')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'what planet is this?" "You have some sort of evidence, uh.....I have some paper'

In [6]:
crea_texto(generador, 'humans are')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'humans are generally considered more difficult to access due to its poor resistance to the drug, which involves a'

In [8]:
crea_texto(generador, 'el monstruo de las galletas')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'el monstruo de las galletas\ny lo ha mencionado\n\nEnglish'

In [9]:
crea_texto(generador, 'molt be noi')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'molt be noi ausser kryptopulismatektor,\nEt'

In [10]:
crea_texto(generador, 'rellampagos asgaya')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'rellampagos asgaya de los bienes que oculta más de los pa'

In [11]:
crea_texto(generador, 'humans are', 100000)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'humans are less likely to be found among those who have had no experience in a given topic (e'

In [13]:
#help(generador)

In [15]:
crea_texto(generador, 'what the fuck asshole')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'what the fuck asshole here is?"\n\n"Fuck yeah, man."\n\nAs I was'